In [12]:
import numpy as np
import random

In [203]:
class VacuumCleaner():
    def __init__(self, size=5, sym_empty='_ ', sym_trash='* ',
                 sym_vacuum='v '):
        self.size = size
        self.sym_empty = sym_empty
        self.sym_trash = sym_trash
        self.sym_vacuum = sym_vacuum
        self.mapping = np.vectorize(lambda s: self.sym_empty if s == 0
                                    else self.sym_trash if s == 1
                                    else self.sym_vacuum)
        
        self.field = np.zeros((self.size, self.size))
        self.field[1,1] = 1
        
    def print_field(self):
        print('\n'.join(''.join(str(cell) for cell in row)
                        for row in self.mapping(self.field)))
    
    def draw_field(self):
        return
    
    def run_process(self, iterations=100):
        for i in range(iterations):
            print('iteration {:>3d}: '.format(i))
            index = np.random.randint(self.size, size=2)
            print(index)
            print(self.field[index])
            print(self.field[index[0], index[1]])

In [204]:
v = VacuumCleaner()

In [205]:
v.run_process(2)

iteration   0: 
[4 1]
[[ 0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]]
0.0
iteration   1: 
[1 1]
[[ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]]
1.0
